In [1]:
import pandas as pd
import numpy as np
from pandas import DataFrame, Series

path = "refFlat.txt"
path_fa = "result.fa"

In [2]:
def read_fasta(file_path):
    sequences = {}
    seq_id = None
    current_seq = ""

    with open(file_path, "r") as file:
        for line in file:
            line = line.strip()

            if line.startswith(">"):  
                if seq_id is not None:
                    sequences[seq_id] = current_seq
                    current_seq = ""
                seq_id = line[1:]  
            else:
                current_seq += line

        if seq_id is not None and current_seq != "":
            sequences[seq_id] = current_seq

    return sequences

In [3]:
import re

def split_by_space(text):
    return re.split('\s+', text)

In [4]:
def gene_data(text_list):
    contents =[]
    for i in range(len(text_list)-1):
        contents.append(text_list[i])
    return contents

In [5]:
def result(path):
    with open(path, "r", encoding="utf-8") as file:
        lines =file.readlines()
    result = []
    for line in lines:
        line =split_by_space(line)
        contents=gene_data(line)
        result.append(contents)
    return result
    

In [6]:
gene_data =result(path)
col_names =['geneName','name','chrom','strand','txStart','txEnd','cdsStart','cdsEnd','exonCount','exonStarts','exonEnds']
gene_df=pd.DataFrame(gene_data,columns=col_names)

In [7]:
base_data = read_fasta(path_fa)

In [8]:
def target_base_CT_minus(seq):
    match = re.findall('(?=(\w{20}GG))', seq)
    target_list=[]
    for i in range(len(match)):
        target = match[i][0:3]
        if target =='CAA'or target =='CAG'or target =='CGA':
            target_list.append(True)
        
        else:
            target_list.append(False)
    result =[match[s] for s in range(len(match)) if target_list[s]]
    result = list(dict.fromkeys(result))
    return result

In [9]:
def target_base_AG_minus(seq):
    match = re.findall('(?=(CC\w{20}))', seq)
    target_list=[]
    for i in range(len(match)):
        target = match[i][-3:]
        if target =='TGG':
            target_list.append(True)
        
        else:
            target_list.append(False)
    result =[match[s] for s in range(len(match)) if target_list[s]]
    result = list(dict.fromkeys(result))
    return result

In [10]:
gene_df

,geneName,name,chrom,strand,txStart,txEnd,cdsStart,cdsEnd,exonCount,exonStarts,exonEnds
0,Xkr4,NM_001011874,chr1,-,3284704,3741721,3286244,3741571,3,"3284704,3491924,3740774,","3287191,3492124,3741721,"
1,Rp1,NM_001370921,chr1,-,4190088,4430526,4190226,4423048,23,"4190088,4212834,4218034,4218834,4234077,424042...","4190296,4212989,4218186,4218967,4234164,424062..."
2,Rp1,NM_001195662,chr1,-,4361068,4479464,4363203,4479410,4,"4361068,4422132,4422424,4479392,","4363235,4422304,4423060,4479464,"
3,Rp1,NM_011283,chr1,-,4413729,4430537,4414822,4423048,4,"4413729,4422132,4422424,4430422,","4420314,4422304,4423060,4430537,"
4,Sox17,NM_001289464,chr1,-,4561150,4567577,4561938,4563629,4,"4561150,4563322,4563994,4566513,","4562891,4563713,4564086,4567577,"
...,...,...,...,...,...,...,...,...,...,...,...
47736,Kcnc3,NR_169735,chr7,+,44239875,44254175,44254175,44254175,6,"44239875,44244584,44247804,44248229,44250097,4...","44241182,44245692,44247999,44248357,44250352,4..."
47737,Kcnc3,NR_169736,chr7,+,44239875,44254175,44254175,44254175,6,"44239875,44244584,44247804,44248229,44250292,4...","44241182,44245692,44247999,44248357,44250352,4..."
47738,Kcnc3,NR_169737,chr7,+,44239875,44254175,44254175,44254175,6,"44239875,44244584,44247807,44248229,44250292,4...","44241182,44245692,44247999,44248357,44250352,4..."
47739,Ttn,NM_001385708,chr2,-,76534325,76812890,76535374,76810526,349,"76534325,76536013,76536783,76537040,76538502,7...","76535673,76536316,76536937,76537732,76538659,7..."


In [19]:
def transcription(seq):
    complementary = {'A':'T','T':'A','G':'C','C':'G'}
    rev_seq = seq[::-1]
    result = []
    for base in rev_seq:
        base =base.upper()
        result.append(complementary[base])
    
    return ''.join(result)

In [20]:
def where_start_codon(name):
    data = gene_df[gene_df['name']==name]
    cdsStart =int(data['cdsEnd'].to_list()[0])
    start = data['exonStarts'].to_list()[0]
    end = data['exonEnds'].to_list()[0]#最後に空白があるからその対処
    start = list(start.split(','))
    end = list(end.split(','))
    for s in range(len(start)-1):
        if (int(start[s]) <= cdsStart<=int(end[s])):
            exon_num =s
    return exon_num


In [21]:
def trans_exon_start(name,sequence_data):
 
    
    
    data = gene_df[gene_df['name']==name]
    
    set_num =int(data['txStart'].to_list()[0])
    start = data['exonStarts'].to_list()[0]
    end = data['exonEnds'].to_list()[0]#最後に空白があるからその対処
    exon_seq_list = []
    start_list = [num for num in start.split(",")]
    end_list = [num for num in end.split(",")]
    
    for s in range(len(start_list)-1):
        start_num = int(start_list[s])- set_num
        
        end_num= int(end_list[s]) - set_num
    
        
        exon_seq =sequence_data[start_num:end_num]
        exon_seq_list.append(exon_seq)

    result = ''.join(exon_seq_list)
    
    exon_num=where_start_codon(name)
    
    exon_count = data['exonCount']
    exon_start = int(exon_count)-1-exon_num
    
    if exon_start == 0:
        result = result[:- (int(end_list[len(start_list)-2])-int(data['cdsEnd'].to_list()[0]))]
        

    else:
        for s in range(exon_start):
            exon_start_num =0
            exon_start_num += int(end_list[-2-s])-int(start_list[-2-s])
            

        
        exon_start_num += int(end_list[-2-exon_start])-int(data['cdsEnd'].to_list()[0])
        result = result[:-(exon_start_num)]
        
    
    return result
    
    

In [28]:
#transcription(trans_exon_start('NM_011283',base_data['NM_011283::chr1:4413729-4430537']))

In [29]:
def end_codon(cds_seq):
    matches = re.finditer('(?=(CAA)|(?=(CAG))|(?=(CGA))|(?=(TGG)))', cds_seq)
    result=[]
    for match in matches:
        start_pos = match.start() 
        
        if (start_pos % 3) == 0:
            result.append(start_pos)
    return result

In [30]:
end_codon(transcription(trans_exon_start('NM_011283',base_data['NM_011283::chr1:4413729-4430537'])))

[54,
 135,
 261,
 375,
 408,
 540,
 561,
 573,
 582,
 636,
 711,
 762,
 828,
 924,
 981,
 1014,
 1047,
 1191,
 1194,
 1224,
 1236,
 1239,
 1266,
 1296,
 1305,
 1311,
 1362,
 1413,
 1422,
 1524,
 1593,
 1629,
 1983,
 1986,
 1989,
 2082,
 2097,
 2139,
 2244,
 2259,
 2292,
 2325,
 2337,
 2397,
 2436,
 2463,
 2553,
 2562,
 2604,
 2700,
 2703,
 2709,
 2763,
 2889,
 2907,
 2976,
 3036,
 3114,
 3141,
 3162,
 3249,
 3258,
 3273,
 3348,
 3393,
 3555,
 3630,
 3750,
 3762,
 3804,
 3864,
 4080,
 4104,
 4170,
 4332,
 4395,
 4401,
 4461,
 4506,
 4686,
 4764,
 4872,
 4902,
 4941,
 4962,
 4989,
 5001,
 5127,
 5151,
 5319,
 5403,
 5496,
 5556,
 5595,
 5637,
 5667,
 5688,
 5739,
 5745,
 5772,
 5862,
 5898,
 5943,
 6036,
 6114,
 6141,
 6165,
 6189,
 6270,
 6351,
 6459,
 6474,
 6483,
 6489,
 6522,
 6552,
 6600,
 6609,
 6774,
 6789,
 6864,
 6990,
 7056,
 7077,
 7098,
 7161,
 7188]

In [53]:
def add_num_list(name):#修正が必要
    data = gene_df[gene_df['name']==name]
    set_num =int(data['cdsStart'].to_list()[0])
    start = data['exonStarts'].to_list()[0]
    end = data['exonEnds'].to_list()[0]#最後に空白があるからその対処
    start = list(start.split(','))
    end = list(end.split(','))
    add_list=[]
    num = 0
    for s in range(len(start)-1):
        element_list=[]
        element_list.append(num)
        
        num += int(end[s])-int(start[s])
        
        element_list.append(num)
        add_list.append(element_list)
        
    return add_list

In [55]:
gene_df[gene_df['name']=='NM_011283']

,geneName,name,chrom,strand,txStart,txEnd,cdsStart,cdsEnd,exonCount,exonStarts,exonEnds
3,Rp1,NM_011283,chr1,-,4413729,4430537,4414822,4423048,4,"4413729,4422132,4422424,4430422,","4420314,4422304,4423060,4430537,"


In [56]:
len(transcription(trans_exon_start('NM_011283',base_data['NM_011283::chr1:4413729-4430537'])))

7381

In [54]:
add_num_list('NM_011283')

[[0, 6585], [6585, 6757], [6757, 7393], [7393, 7508]]